In [1]:
import os
import numpy as np
import dill
import pickle
from bottleneck import push
from fancyimpute import IterativeImputer
from miss_utils import *

In [2]:
data_path = 'extracts/'
with open(os.path.join(data_path, '3d_tensor_set_a.pkl'), 'rb') as f:
    origional_a = pickle.load(f)
    
with open(os.path.join(data_path, '3d_tensor_set_b.pkl'), 'rb') as f:
    origional_b = pickle.load(f)    

with open(os.path.join(data_path, '3d_tensor_set_c.pkl'), 'rb') as f:
    origional_c = pickle.load(f)
    
origional = np.vstack([origional_a,origional_b,origional_c]) 
mask=~np.isnan(origional)*1

In [3]:

mins = []
maxs = []
flatten= origional.reshape(origional.shape[0]*origional.shape[1], origional.shape[2])

for i in range(origional.shape[2]):

        mins.append(np.nanmin(flatten[:,i]))

        maxs.append(np.nanmax(flatten[:,i]))

normalized =normalize(origional,mins,maxs)

In [4]:

# cols	Sample	feat
# 0	ALP	98.363715	57.525000
# 4	Lactate	95.886458	45.200000
# 9	Albumin	98.743229	59.416667
# 10	ALT	98.316840	56.558333
# 12	SaO2	95.985590	55.275000
# 14	AST	98.316667	56.550000
# 15	Bilirubin	98.299826	56.550000
# 24	Cholesterol	99.827431	92.100000
# 26	TroponinT	98.915451	78.025000
# 28	TroponinI	99.796181	95.291667

Patient level introduce

In [5]:
seed = 1234
select_cols= [1,2,20,27,31]
#select_cols= [15,10,9,14,26]

new_tensor, miss_indices, indicating_mask=introduce_miss_cols(normalized,mask, 0.5, 1243, select_cols)
LO =push(new_tensor, axis=1)
LOCV= np.where(np.isnan(LO), 0, LO)
zero= np.where(np.isnan(new_tensor), 0, new_tensor)
means, stds = [],[]
flatten= new_tensor.reshape(origional.shape[0]*origional.shape[1], 35)
for i in range(35):
    means.append(np.nanmean(flatten[:,i]))
    stds.append(np.nanstd(flatten[:,i]))
mean_imputed =mean_fill(new_tensor, means)
mean_imputed =mean_fill(new_tensor, means)
mask_miss=~np.isnan(new_tensor)*1
imputed=prepare_fills(new_tensor, means)
IMM= create_individualized_missingness_mask(mask_miss)

In [6]:
cal_missing_rate(new_tensor)

0.8510131448412699

In [7]:
# with open('extracts/X_person_0.5_'+str(seed)+'_col_most.pkl', 'wb') as outfile:
#     dill.dump(new_tensor, outfile, pickle.HIGHEST_PROTOCOL)  
# with open('extracts/new__miss_oo_person_0.5_'+str(seed)+'_col_most.pkl', 'wb') as outfile:
#     dill.dump(zero, outfile, pickle.HIGHEST_PROTOCOL) 
# with open('extracts/new__miss_0.5_person_back_forward_'+str(seed)+'_col_most.pkl', 'wb') as outfile:
#     dill.dump(imputed, outfile, pickle.HIGHEST_PROTOCOL) 
# with open('extracts/mask_miss__person_0.5_'+str(seed)+'_col_most.pkl', 'wb') as outfile:
#     dill.dump(mask_miss, outfile, pickle.HIGHEST_PROTOCOL)  
# with open('extracts/IMM_person_0.5_'+str(seed)+'_col_most.pkl', 'wb') as outfile:
#     dill.dump(IMM, outfile, pickle.HIGHEST_PROTOCOL)  

In [8]:
mice_impute = IterativeImputer()
flatten= new_tensor.reshape(new_tensor.shape[0]*new_tensor.shape[1], new_tensor.shape[2])
MICE = mice_impute.fit_transform(flatten)
MICE=MICE.reshape(new_tensor.shape[0], new_tensor.shape[1], new_tensor.shape[2])

In [9]:
experiment_name = "IMM_person_1234_05_most_col"
saved_path = 'data/'
observed_only_real = np.load(saved_path + experiment_name + '.npz')["observed_only_real"]
observed_only_rec = np.load(saved_path + experiment_name + '.npz')["observed_only_rec"]
IMM_real = np.load(saved_path + experiment_name + '.npz')["IMM_real"]
IMM_rec = np.load(saved_path + experiment_name + '.npz')["IMM_rec"]
with open(os.path.join('SAITS_person_0.5_1234_col.pkl'), 'rb') as f:
           SAITS = pickle.load(f)    
with open(os.path.join('BRITS_person_0.5_1234_col.pkl'), 'rb') as f:
            BRITS = pickle.load(f)  

In [10]:
imputed_ours =(mask_miss * (zero)+ ((1-mask_miss) *IMM_rec))

In [11]:
X_intact= np.where(np.isnan(normalized), 0, normalized)

In [12]:
MICE =MICE.copy()

In [14]:
# flat= np.zeros((12000,48,35)).reshape(-1)
# flat[miss_indices] = 1
# indicating_mask=flat.reshape(12000,48,35)


In [15]:
print("MAE")
print("----------------------")
get_patient_level_MAE_array(LOCV, normalized, indicating_mask, "LOCF") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(imputed_ours, normalized, indicating_mask, "IGNITE") # clculate mean absolute error on the ground truth (artificially-missing values)

MAE
----------------------
LOCF 0.0331666483423677 0.015123893346116524
mean 0.03775553780912004 0.015719988040577663
mice 0.0316631062622157 0.013212362442140186
BRITS 0.017446398400448815 0.007133646518165361
SAITS 0.019861214620670822 0.007505929147288343
IGNITE 0.0939157434780144 0.044120796233078054


In [16]:
print("RMSE")
print("----------------------")
get_patient_level_RMSE_array(LOCV, normalized, indicating_mask, "LOCF") 
get_patient_level_RMSE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

RMSE
----------------------
LOCF 0.06627642262519275 0.03165544659760638
mean 0.05091675177483586 0.020821288937780718
mice 0.04444632279006252 0.018971898545788988
BRITS 0.027156773673542925 0.014914918288044537
SAITS 0.029680110940822582 0.01456770580739196
IGNITE 0.11738609785773804 0.052309127510365434


In [17]:
print("MSE")
print("----------------------")
get_patient_level_MSE_array(LOCV, normalized, indicating_mask, "LOCF") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(imputed_ours, normalized, indicating_mask, "IGNITE") # calculate mean absolute error on the ground truth (artificially-missing values)

MSE
----------------------
LOCF 0.005394631495287069 0.004928006013963253
mean 0.003026041684330799 0.00272463510231127
mice 0.002335408543990141 0.002583505662761832
BRITS 0.0009599451438950796 0.002041077389720743
SAITS 0.0010931270379502581 0.0017828134786266286
IGNITE 0.01651574079116212 0.014469133532862305


Calculate Population level sum

In [18]:
get_population_RMSE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_RMSE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

LOCF 0.07535946823311
mean 0.05583608587740615
mice 0.04805691557191138
BRITS 0.0316869179889283
SAITS 0.03430170486683019
IGNITE 0.13649728780721018


In [19]:
get_population_MSE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MSE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

LOCF 0.005679049452377116
mean 0.0031176684861090745
mice 0.0023094671342858185
BRITS 0.001004060771637068
SAITS 0.0011766069567711224
IGNITE 0.01863150957872437


In [20]:

#         return lib.sum(lib.abs(inputs - target) * mask) / (lib.sum(mask) + 1e-9)
get_population_MAE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MAE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(imputed_ours, normalized, indicating_mask, "IGNITE") # c

LOCF 0.03512555191230592
mean 0.038756004487510674
mice 0.031546856330048285
BRITS 0.018019893155283167
SAITS 0.02099381447311427
IGNITE 0.10246367619965814


get weighted population sum

In [28]:
get_population_MAE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MAE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_populatiaon_MAE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

LOCF 0.035125551911875996
mean 0.038756004487021295
mice 0.0315468563296229
BRITS 0.01801989315505699


NameError: name 'get_populatiaon_MAE_weighted' is not defined

In [ ]:
get_population_RMSE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_RMSE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [ ]:
get_population_MSE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MSE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [25]:
imputed_ours[1,:,1]

array([0.165735424, 0.193318367, 0.293333333, 0.206881613, 0.203629792,
       0.293333333, 0.293333333, 0.293333333, 0.198698521, 0.199368238,
       0.266666667, 0.201000512, 0.266666667, 0.266666667, 0.266666667,
       0.199797153, 0.266666667, 0.266666667, 0.191591978, 0.194769919,
       0.18991977 , 0.27       , 0.266666667, 0.192242503, 0.266666667,
       0.191875398, 0.186246723, 0.191126555, 0.266666667, 0.191593587,
       0.185452282, 0.190628111, 0.187229365, 0.226666667, 0.185252488,
       0.190384179, 0.23       , 0.191483378, 0.185270041, 0.190238208,
       0.187246412, 0.191506863, 0.185381114, 0.236666667, 0.24       ,
       0.191456676, 0.233333333, 0.216666667])

In [27]:
indicating_mask[1,:,1]

array([0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0.])